In [1]:
from torchsummary import summary
from python_lib.modules import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


In [2]:
input_feats = torch.rand([1,2,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (2,64))
print(input_feats_np)
dim = input_feats_np.shape

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"ecapainput.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[2.14627564e-01 5.19425094e-01 9.03343558e-02 2.71279633e-01
  8.00896764e-01 3.75588000e-01 5.82802892e-01 1.87831163e-01
  2.32388556e-01 9.17293429e-02 4.29272354e-01 6.78330421e-01
  2.23407269e-01 8.05796027e-01 4.49194074e-01 3.51586282e-01
  9.33311582e-02 5.18459916e-01 4.18873072e-01 7.81540215e-01
  9.67993081e-01 3.93763304e-01 5.37199974e-02 5.79132140e-01
  5.18599033e-01 7.23363400e-01 2.06866503e-01 9.69597876e-01
  4.67864871e-02 6.78856671e-01 4.34937716e-01 3.22561502e-01
  7.60028422e-01 9.14776325e-02 3.87083709e-01 1.37053192e-01
  1.45381272e-01 4.21477139e-01 7.02052832e-01 7.01632261e-01
  2.86228240e-01 8.85494828e-01 4.20499444e-01 2.14118063e-01
  6.00423813e-02 1.92389250e-01 5.15105903e-01 7.87610948e-01
  9.68760490e-01 5.26328802e-01 4.35471177e-01 8.07291269e-01
  2.04085708e-02 6.19444549e-01 1.12393379e-01 4.90337968e-01
  2.40671575e-01 8.64619732e-01 9.69111919e-03 7.93832123e-01
  5.46234250e-01 5.91592193e-01 7.78553843e-01 8.69101167e-01]
 [4.407

In [3]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")

model_ = BlockSave(model.return_layers(), "fullecapa", "ECAPAweights")
model_.save()

initialblock = BlockSave(model.blocks[0].return_layers(), "initialblock", "ECAPAweights")
initialblock.save()

seres2_1 = BlockSave(model.blocks[1].return_layers(), "seres2_1", "ECAPAweights")
seres2_1.save()
seres2_2 = BlockSave(model.blocks[2].return_layers(), "seres2_2", "ECAPAweights")
seres2_2.save()
seres2_3 = BlockSave(model.blocks[3].return_layers(), "seres2_3", "ECAPAweights")
seres2_3.save()

mfa = BlockSave(model.mfa.return_layers(), "mfa", "ECAPAweights")
mfa.save()

asp = BlockSave(model.asp.return_layers(), "asp", "ECAPAweights")
asp.save()

# asp_bn = BlockSave(model.asp_bn.return_layers(), "asp_bn", "ECAPAweights")
# asp_bn.save()

fc = BlockSave([model.final[1]], "fc", "ECAPAweights")
fc.save()

In [7]:
model(input_feats)

RuntimeError: Given groups=1, weight of size [8, 2, 5], expected input[1, 8, 68] to have 2 channels, but got 8 channels instead

In [5]:
input_feats.mean(dim=2, keepdim=True).shape

torch.Size([1, 8, 1])

In [6]:
import math

def get_padding_elem(L_in: int, stride: int, kernel_size: int, dilation: int):
    """This function computes the number of elements to add for zero-padding.

    Arguments
    ---------
    L_in : int
    stride: int
    kernel_size : int
    dilation : int

    Returns
    -------
    padding : int
        The size of the padding to be added
    """
    if stride > 1:
        padding = [math.floor(kernel_size / 2), math.floor(kernel_size / 2)]

    else:
        L_out = (
            math.floor((L_in - dilation * (kernel_size - 1) - 1) / stride) + 1
        )
        padding = [
            math.floor((L_in - L_out) / 2),
            math.floor((L_in - L_out) / 2),
        ]
    return padding

get_padding_elem(64,1,5,1)

[2, 2]